In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (10,8) 

# Configuración warnings
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns=None

En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.

In [3]:
df=pd.read_csv('../datos/salary_normalizado.csv', index_col=0)
df.head(2) #Hacemos una vista previa de nuestro DataFrame

work_year experience_level employment_type              job_title  \
0       2020      Entry-Level       Full-Time           Data Analyst   
1       2020        Mid-Level       Full-Time  Business Data Analyst   

   salary_in_usd employee_residence  remote_ratio company_location  \
0          72000                 US           100               US   
1         135000                 US           100               US   

  company_size   salary_box  
0            L  4631.904560  
1            L  7311.922975

Hasta ahora habéis estado evaluando las características de vuestro set de datos y habéis hecho una gran exploración, es el momento de hacer vuestro primer ANOVA! En el ejercicio de hoy tendréis que hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.

Vamos a utilizar las variables que cumplen la asunción de homocedasticidad y normalidad (ninguna cumple la estandarización por no tenemos variables númericas)

In [4]:
# iniciamos nuestro anova. Recordemos que la variable respuesta es el precio de las casas (median_house_value)
lm = ols('salary_box ~ work_year  + experience_level + employee_residence + company_location' , data=df).fit()
sm.stats.anova_lm(lm)

df        sum_sq       mean_sq          F        PR(>F)
experience_level      3.0  4.506350e+08  1.502117e+08  70.445125  1.737270e-34
employee_residence    5.0  2.888119e+08  5.776237e+07  27.088958  1.366565e-22
company_location      5.0  1.418017e+07  2.836034e+06   1.330022  2.512889e-01
work_year             1.0  2.965618e+04  2.965618e+04   0.013908  9.062008e-01
Residual            299.0  6.375642e+08  2.132322e+06        NaN           NaN

In [5]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             salary_box   R-squared:                       0.542
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     32.12
Date:                Thu, 29 Sep 2022   Prob (F-statistic):           1.85e-44
Time:                        12:57:44   Log-Likelihood:                -2701.2
No. Observations:                 311   AIC:                             5426.
Df Residuals:                     299   BIC:                             5471.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                           -3.483e+04   3.35e+05     -0.104      0.917   -6.95e+05    6.25e+05
experience_level[T.Executive Level]  3499.0530    727.968      4.807      0.000    2066.463    4931.643
experience_level[T.Mid-Level]        1092.5298    315.017      3.468      0.001     472.599    1712.460
experience_level[T.Senior]           2022.4765    329.480      6.138      0.000    1374.082    2670.871
employee_residence[T.ES]             2137.4221   1121.615      1.906      0.058     -69.838    4344.682
employee_residence[T.FR]             -317.1116    301.950     -1.050      0.294    -911.329     277.105
employee_residence[T.GB]               48.0388    231.122      0.208      0.835    -406.794     502.871
employee_residence[T.IN]            -4296.2225   1112.722     -3.861      0.000   -6485.981   -2106.464
employee_residence[T.US]             1467.0345    734.708      1.997      0.047      21.181    2912.888
company_location[T.ES]              -3169.8770   1190.277     -2.663      0.008   -5512.258    -827.496
company_location[T.FR]               -317.1116    301.950     -1.050      0.294    -911.329     277.105
company_location[T.GB]                 48.0388    231.122      0.208      0.835    -406.794     502.871
company_location[T.IN]               2160.9323   1168.880      1.849      0.065    -139.342    4461.206
company_location[T.US]                317.1790    733.040      0.433      0.666   -1125.393    1759.751
work_year                              19.0238    165.929      0.115      0.909    -307.513     345.561
==============================================================================
Omnibus:                        3.470   Durbin-Watson:                   1.858
Prob(Omnibus):                  0.176   Jarque-Bera (JB):                3.411
Skew:                           0.155   Prob(JB):                        0.182
Kurtosis:                       3.409   Cond. No.                     5.34e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.46e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Interpretación de los resultados de summary


Una vez obtenidos los resultados de summary, si nos fijamos en el pvalor obtenido para las distintas variables categóricas, vemos que hay algunas que no nos estan afectando a nuestra variable respuesta (hemos obtenido un pvalor > 0.05), como son, por ejemplo company_location y work_year, que de media vemos que superan los 0.05 puntos en el pvalor. De esta forma, decidimos no seguir con estas variables para el modelo de regresión lineal, y coger solo las que han obtenido de media un pvalor < 0.05, y que nos indican que si que afectan a nuestras variable respuesta. Estas son:

 - experience_level
 - employee_residence

Guardamos el DF con las variables predictoras categóricas correctas y nuestra variable respuesta

In [7]:
df.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary_in_usd', 'employee_residence', 'remote_ratio',
       'company_location', 'company_size', 'salary_box'],
      dtype='object')

In [8]:
df_modelo = df[['salary_box','experience_level','employee_residence']]

In [9]:
df_modelo.head(2)

salary_box experience_level employee_residence
0  4631.904560      Entry-Level                 US
1  7311.922975        Mid-Level                 US

In [10]:
df_modelo.to_csv('../datos/salaries_modelo.csv')

📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer cuando nos encontramos en esta situación.